In [1]:
from conf import databases, url, data_dir
from file_utils import create_folder
from dnn_helper import *

import numpy
import random
import time

import lasagne
from matplotlib import pyplot as plt
import matplotlib as mpl

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device gpu failed:
cublasCreate() returned this error 'the CUDA Runtime initialization failed'


MemoryError: Error allocating 40000 bytes of device memory (CNMEM_STATUS_NOT_INITIALIZED).

In [3]:
fs = 128
trained_fs = 360
min_bpm = 10
max_bpm = 350
min_gap = fs*60/min_bpm
max_gap = fs*60/max_bpm

# NN params
segment_size = 5000
segment_step = 2500
y_delay = 0

train_perc = 99
test_perc = 100 - train_perc

# Border suppression for evaluation
left_border  = 50 # samples
right_border = 50 # samples

In [7]:
class NN:
    def __init__(self, model_builder, dim, params):
        self.t_in = tensor.ftensor3('inputs')  #  =X     float64
        self.t_out = tensor.imatrix('targets') # =Y_true int32
        self.input_shape = (None, dim, segment_size,)
        self.output_shape  = (None, dim, segment_size,)
        self.model = model_builder(self.t_in, dim=dim, shape=self.input_shape)
        self.init_funs(self.model)
        self.params = params
    
    def init_funs(self, model):
        test_pred = get_output(self.model, deterministic=True)
        test_loss = my_loss(self.model, test_pred, self.t_out, False)
        test_loss_with_reg = my_loss(self.model, test_pred, self.t_out, True)
        test_acc  = tensor.mean(tensor.eq(tensor.argmax(test_pred, axis=1), self.t_out), dtype=config.floatX)
        
        self.eval_fn  = function([self.t_in, self.t_out], [test_loss, test_loss_with_reg, test_acc], allow_input_downcast=True)
        self.evaluate = function([self.t_in], get_output(self.model, self.t_in), allow_input_downcast=True)
        
        pred = get_output(self.model)
        loss_with_reg = my_loss(self.model, pred, self.t_out, True)
        params = get_all_params(self.model, trainable=True)
        updates = adam(loss_with_reg, params=params, learning_rate=0.0001)
        self.train_fn = function([self.t_in, self.t_out], loss_with_reg, updates=updates, allow_input_downcast=True)
    
    def train(self, train_exs, test_exs, num_epochs, examples_by_epoch, stop_accuracy=100.0, info=False):
        trainN, testN = len(train_exs), len(test_exs)
        print('Training on {} examples, testing on {} examples.'.format(trainN, testN))
        print("Starting training...")
        
        ex_count = 0
        train_losses = []
        rps = []
        for epoch in range(1, num_epochs+1):
            if epoch < 3:
                train_losses = []
            start_time = time.time()
            print('Epoch {}/{} running...'.format(epoch, num_epochs), end='')
            train_loss = 0
            train_loss_with_reg = 0
            invalid_example = 0
            for z in range(examples_by_epoch):
                db, i, j = train_exs[ex_count%trainN]
                XY = load_steps(db, i, *self.params)[j]
                x, y = numpy.reshape(XY[0], (1, 1, 5000)).astype('float16'), numpy.reshape(XY[1], (1, 5000)).astype('float16')
                ex_count += 1
                if numpy.sum(y) == 0.:
                    invalid_example += 1
                    continue
                tmp_loss = self.train_fn(x, y)
                train_loss += tmp_loss
                train_losses.append(train_loss/(z+1-invalid_example))
            print('Done in {:.3f}s!'.format(time.time() - start_time))
            print("  - training loss:\t\t{:.6f}".format(train_loss / (examples_by_epoch-invalid_example)))
            
            # Eval on examples:
            test_loss = 0
            test_reg_loss = 0
            test_acc = 0
            invalid_example = 0
            for k in range(testN):
                db, i, j = train_exs[k]
                XY = load_steps(db, i, *self.params)[j]
                x, y = numpy.reshape(XY[0], (1, 1, 5000)).astype('float16'), numpy.reshape(XY[1], (1, 5000)).astype('float16')
                if numpy.sum(y) == 0.:
                    invalid_example += 1
                    continue
                tmp_loss, tmp_reg_loss, tmp_acc = self.eval_fn(x, y)
                test_loss += tmp_loss
                test_reg_loss += (tmp_reg_loss-tmp_loss)
                test_acc  += tmp_acc
            N = (testN-invalid_example)
            print("  - test loss:\t\t\t{:.6f} | {:.6f} | {:.6f}".format(test_loss / N, test_reg_loss / N, (test_loss+test_reg_loss) / N))
            acc = test_acc / N * 100
            print("  - test accuracy:\t\t{:.4f} %".format(acc))
            plt.plot(train_losses, color='r')
            plt.plot(rps, color='g')
            plt.show()
            eval_model(test_exs, self.evaluate, min_gap, max_gap, left_border, right_border, *self.params,
                       plot_examples=True, nb=3, nearest_fpr=0.01, threshold=0.95, eval_margin=10)
            save_model(model_path + '/model-loss{}-epoch{}.sav'.format(test_loss / N, epoch), self.model, get_all_param_values(self.model))
            if acc > stop_accuracy:
                break
    
    def eval_data(self, x, fs_target):
        fs_target, y_delay, segment_size, segment_step, normalized_steps = self.params
        

In [8]:
def build_model(input_tensor, dim, shape=None):
    print('Model shape:')
    
    l_in = InputLayer(shape, input_var=input_tensor)
    
    print('  Layer A:')
    a1 = Conv1DLayer(l_in, num_filters=8, filter_size=(3), nonlinearity=rectify, pad='same')
    print('    a1:', get_output_shape(a1))
    a2 = Conv1DLayer(l_in, num_filters=32, filter_size=(5), nonlinearity=rectify, pad='same')
    print('    a2:', get_output_shape(a2))
    a3 = Conv1DLayer(l_in, num_filters=64, filter_size=(9), nonlinearity=rectify, pad='same')
    print('    a3:', get_output_shape(a3))
    a4 = Conv1DLayer(l_in, num_filters=64, filter_size=(15), nonlinearity=rectify, pad='same')
    print('    a4:', get_output_shape(a4))
    a5 = Conv1DLayer(l_in, num_filters=64, filter_size=(25), nonlinearity=rectify, pad='same')
    print('    a5:', get_output_shape(a5))
    a6 = Conv1DLayer(l_in, num_filters=64, filter_size=(35), nonlinearity=rectify, pad='same')
    print('    a6', get_output_shape(a6))
    a7 = Conv1DLayer(l_in, num_filters=64, filter_size=(45), nonlinearity=rectify, pad='same')
    print('    a7:', get_output_shape(a7))
    a8 = Conv1DLayer(l_in, num_filters=32, filter_size=(89), nonlinearity=rectify, pad='same')
    print('    a8:', get_output_shape(a8))
    a9 = Conv1DLayer(l_in, num_filters=32, filter_size=(119), nonlinearity=rectify, pad='same')
    print('    a9:', get_output_shape(a9))
    a10 = Conv1DLayer(l_in, num_filters=16, filter_size=(149), nonlinearity=rectify, pad='same')
    print('    a10:', get_output_shape(a10))
    a11 = Conv1DLayer(l_in, num_filters=16, filter_size=(199), nonlinearity=rectify, pad='same')
    print('    a11:', get_output_shape(a11))
    a12 = Conv1DLayer(l_in, num_filters=16, filter_size=(299), nonlinearity=rectify, pad='same')
    print('    a12:', get_output_shape(a12))
    al = ConcatLayer([a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12], axis=1)
    print('   al:', get_output_shape(al))

    ad = DropoutLayer(al, p=0.5)

    print('  Layer B:')
    b1 = Conv1DLayer(ad, num_filters=8, filter_size=3, nonlinearity=rectify, pad='same')
    print('    b1:', get_output_shape(b1))
    b2 = Conv1DLayer(ad, num_filters=64, filter_size=5, nonlinearity=rectify, pad='same')
    print('    b2:', get_output_shape(b2))
    b3 = Conv1DLayer(ad, num_filters=64, filter_size=9, nonlinearity=rectify, pad='same')
    print('    b3:', get_output_shape(b3))
    b4 = Conv1DLayer(ad, num_filters=64, filter_size=19, nonlinearity=rectify, pad='same')
    print('    b4:', get_output_shape(b4))
    b5 = Conv1DLayer(ad, num_filters=64, filter_size=29, nonlinearity=rectify, pad='same')
    print('    b5:', get_output_shape(b5))
    b6 = Conv1DLayer(ad, num_filters=64, filter_size=49, nonlinearity=rectify, pad='same')
    print('    b6:', get_output_shape(b6))
    bl = ConcatLayer([b1, b2, b3, b4, b5, b6], axis=1)
    print('   bl:', get_output_shape(bl))
    
    bd = DropoutLayer(bl, p=0.5)

    print('  Layer C:')
    c1 = Conv1DLayer(bd, num_filters=8, filter_size=3, nonlinearity=rectify, pad='same')
    print('    c1:', get_output_shape(c1))
    c2 = Conv1DLayer(bd, num_filters=32, filter_size=5, nonlinearity=rectify, pad='same')
    print('    c2:', get_output_shape(c2))
    c3 = Conv1DLayer(bd, num_filters=32, filter_size=9, nonlinearity=rectify, pad='same')
    print('    c3:', get_output_shape(c3))
    c4 = Conv1DLayer(bd, num_filters=32, filter_size=15, nonlinearity=rectify, pad='same')
    print('    c4:', get_output_shape(c4))
    c5 = Conv1DLayer(bd, num_filters=32, filter_size=19, nonlinearity=rectify, pad='same')
    print('    c5:', get_output_shape(c5))
    cl = ConcatLayer([c1, c2, c3, c4, c5])
    print('   cl:', get_output_shape(cl))
    
    cd = DropoutLayer(cl, p=0.4)
    
    print('  Layer D:')
    d1 = Conv1DLayer(cd, num_filters=8, filter_size=3, nonlinearity=rectify, pad='same')
    print('    d1:', get_output_shape(d1))
    d2 = Conv1DLayer(cd, num_filters=32, filter_size=5, nonlinearity=rectify, pad='same')
    print('    d2:', get_output_shape(d2))
    d3 = Conv1DLayer(cd, num_filters=64, filter_size=9, nonlinearity=rectify, pad='same')
    print('    d3:', get_output_shape(d3))
    d4 = Conv1DLayer(cd, num_filters=64, filter_size=15, nonlinearity=rectify, pad='same')
    print('    d4:', get_output_shape(d4))
    d5 = Conv1DLayer(cd, num_filters=64, filter_size=19, nonlinearity=rectify, pad='same')
    print('    d5:', get_output_shape(d5))
    dl = ConcatLayer([d1, d2, d3, d4, d5])
    print('   dl:', get_output_shape(dl))
    
    dd = DropoutLayer(dl, p=0.3)
    
    print('  Later E:')
    e1 = Conv1DLayer(dd, num_filters=8, filter_size=3, nonlinearity=rectify, pad='same')
    print('    e1:', get_output_shape(e1))
    e2 = Conv1DLayer(dd, num_filters=16, filter_size=5, nonlinearity=rectify, pad='same')
    print('    e2:', get_output_shape(e2))
    e3 = Conv1DLayer(dd, num_filters=64, filter_size=9, nonlinearity=rectify, pad='same')
    print('    e3:', get_output_shape(e3))
    e4 = Conv1DLayer(dd, num_filters=64, filter_size=15, nonlinearity=rectify, pad='same')
    print('    e4:', get_output_shape(e4))
    el = ConcatLayer([e1, e2, e3, e4])
    print('   el:', get_output_shape(el))
    
    ed = DropoutLayer(el, p=0.2)
    
    print('  Layer F:')
    f1 = Conv1DLayer(ed, num_filters=8, filter_size=3, nonlinearity=rectify, pad='same')
    print('    f1:', get_output_shape(f1))
    f2 = Conv1DLayer(ed, num_filters=16, filter_size=9, nonlinearity=rectify, pad='same')
    print('    f2:', get_output_shape(f2))
    f3 = Conv1DLayer(ed, num_filters=32, filter_size=15, nonlinearity=rectify, pad='same')
    print('    f3:', get_output_shape(f3))
    fl = ConcatLayer([f1, f2, f3])
    print('   fl:', get_output_shape(fl))
    
    fd = DropoutLayer(fl, p=0.1)
    
    print('  Layer G:')
    g1 = Conv1DLayer(fd, num_filters=1, filter_size=3, nonlinearity=sigmoid, pad='same')
    print('    g1:', get_output_shape(g1))
    gl = ConcatLayer([g1])
    print('   gl:', get_output_shape(gl))
    
    return gl


nn = NN(dim=1, model_builder=build_model, params=(360, 5000, 2500, True, True))

Model shape:
  Layer A:
    a1: (None, 8, 5000)
    a2: (None, 32, 5000)
    a3: (None, 64, 5000)
    a4: (None, 64, 5000)
    a5: (None, 64, 5000)
    a6 (None, 64, 5000)
    a7: (None, 64, 5000)
    a8: (None, 32, 5000)
    a9: (None, 32, 5000)
    a10: (None, 16, 5000)
    a11: (None, 16, 5000)
    a12: (None, 16, 5000)
   al: (None, 472, 5000)
  Layer B:
    b1: (None, 8, 5000)
    b2: (None, 64, 5000)
    b3: (None, 64, 5000)
    b4: (None, 64, 5000)
    b5: (None, 64, 5000)
    b6: (None, 64, 5000)
   bl: (None, 328, 5000)
  Layer C:
    c1: (None, 8, 5000)
    c2: (None, 32, 5000)
    c3: (None, 32, 5000)
    c4: (None, 32, 5000)
    c5: (None, 32, 5000)
   cl: (None, 136, 5000)
  Layer D:
    d1: (None, 8, 5000)
    d2: (None, 32, 5000)
    d3: (None, 64, 5000)
    d4: (None, 64, 5000)
    d5: (None, 64, 5000)
   dl: (None, 232, 5000)
  Later E:
    e1: (None, 8, 5000)
    e2: (None, 16, 5000)
    e3: (None, 64, 5000)
    e4: (None, 64, 5000)
   el: (None, 152, 5000)
  Layer F:

In [ ]:
exs = shuffled_examples(360, 5000, 2500, True, True)
train_exs = exs[:int(len(exs)*train_perc/100)]
test_exs  = exs[int(len(exs)*train_perc/100):]
print('Number of examples: {} ({} samples)'.format(len(exs), len(exs)*5000))
print('          Training: {} ({} samples)'.format(len(train_exs), len(train_exs)*5000))
print('           Testing: {} ({} samples)'.format(len(test_exs), len(test_exs)*5000))

Number of examples: 88860 (444300000 samples)
          Training: 87971 (439855000 samples)
           Testing: 889 (4445000 samples)


In [ ]:
nn.train(train_exs, test_exs, num_epochs=200, examples_by_epoch=1000, info=True)

Training on 87971 examples, testing on 889 examples.
Starting training...
Epoch 1/200 running...

In [7]:
_, params = load_model(model_path + '/model-loss0.037631392673427445-epoch1200.sav')
# print(params)
set_all_param_values(nn.model, params)
nn.init_funs(nn.model)

In [ ]:
eval_model(testXY, nn.evaluate, left_border=left_border, right_border=right_border,
           min_gap=min_gap, max_gap=max_gap, plot_examples=True, nb=10,
           nearest_fpr=0.000675, threshold=0.9678147, eval_margin=10)